<a href="https://colab.research.google.com/github/christymanthara/NLPprojectRAGbot/blob/main/NLPprojectipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Get the dataset from Kaggle

In [56]:
!pip install -U -q "langchain"  "bitsandbytes==0.40.2"

In [57]:
! pip install kaggle

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


Create  a key for kaggle and store it in the drive. copy the path to the key to the temporary kaggle folder created above

In [60]:
!cp /content/drive/MyDrive/KAGGLE/kaggle.json ~/.kaggle/

In [61]:
! chmod 600 ~/.kaggle/kaggle.json

In [62]:
!kaggle datasets download -d tentotheminus9/religious-and-philosophical-texts

Dataset URL: https://www.kaggle.com/datasets/tentotheminus9/religious-and-philosophical-texts
License(s): unknown
religious-and-philosophical-texts.zip: Skipping, found more recently modified local copy (use --force to force download)


In [63]:
#now we unzip the downloaded dataset
! unzip religious-and-philosophical-texts.zip

Archive:  religious-and-philosophical-texts.zip
replace 35895-0.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [64]:
!pip install -q -U faiss-cpu tiktoken sentence-transformers

#Task 1: Data Preparation

####Data parsing

In [6]:
!pip install langchain_community

In [7]:
from langchain_community.document_loaders import TextLoader

In [8]:
loader = TextLoader('pg2680.txt')
meditations_data = loader.load()

In [9]:
len(meditations_data)

1

In [10]:
print(meditations_data)

[Document(page_content='\ufeffThe Project Gutenberg EBook of Meditations, by Marcus Aurelius\n\nThis eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre-use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org\n\n\nTitle: Meditations\n\nAuthor: Marcus Aurelius\n\nPosting Date: December 25, 2008 [EBook #2680]\nRelease Date: June, 2001\n\nLanguage: English\n\n\n*** START OF THIS PROJECT GUTENBERG EBOOK MEDITATIONS ***\n\n\n\n\nProduced by J. Boulton\n\n\n\n\n\nMEDITATIONS\n\nBy Marcus Aurelius\n\n\n\n\nCONTENTS\n\n\n     NOTES\n\n     INTRODUCTION\n\n     FIRST BOOK\n\n     SECOND BOOK\n\n     THIRD BOOK\n\n     FOURTH BOOK\n\n     FIFTH BOOK\n\n     SIXTH BOOK\n\n     SEVENTH BOOK\n\n     EIGHTH BOOK\n\n     NINTH BOOK\n\n     TENTH BOOK\n\n     ELEVENTH BOOK\n\n     TWELFTH BOOK\n\n     APPENDIX\n\n     GLOSSARY\n\n\n\n\nOriginal Transcriber\'s Notes

now we are going to split the data

We will use the langchain text splitter for this purpose

https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/character_text_splitter/

In [11]:
import os
from getpass import getpass

In [12]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

In [13]:
with open("pg2680.txt") as f:
    meditations = f.read()

text1 = text_splitter.create_documents([meditations])
print(text1[0])
len(text1)

page_content="\ufeffThe Project Gutenberg EBook of Meditations, by Marcus Aurelius\nThis eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre-use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org\nTitle: Meditations\nAuthor: Marcus Aurelius\nPosting Date: December 25, 2008 [EBook #2680]\nRelease Date: June, 2001\nLanguage: English\n*** START OF THIS PROJECT GUTENBERG EBOOK MEDITATIONS ***\nProduced by J. Boulton\nMEDITATIONS\nBy Marcus Aurelius\nCONTENTS\n     NOTES\n     INTRODUCTION\n     FIRST BOOK\n     SECOND BOOK\n     THIRD BOOK\n     FOURTH BOOK\n     FIFTH BOOK\n     SIXTH BOOK\n     SEVENTH BOOK\n     EIGHTH BOOK\n     NINTH BOOK\n     TENTH BOOK\n     ELEVENTH BOOK\n     TWELFTH BOOK\n     APPENDIX\n     GLOSSARY\nOriginal Transcriber's Notes:\nThis text was scanned by J. Boulton using Textbridge OCR. The Greek\nportions of the text have

509

In [14]:
with open("pg17.txt") as f:
    mormons = f.read()

text2 = text_splitter.create_documents([mormons])
print(text2[2])
len(text2)

page_content='building a tower to get to heaven--Which is to show unto the\nremnant of the House of Israel what great things the Lord hath\ndone for their fathers; and that they may know the covenants of\nthe Lord, that they are not cast off forever--And also to the\nconvincing of the Jew and Gentile that JESUS is the CHRIST, the\nETERNAL GOD, manifesting himself unto all nations--And now, if\nthere are faults they are the mistakes of men; wherefore, condemn\nnot the things of God, that ye may be found spotless at the\njudgment-seat of Christ.\nTRANSLATED BY JOSEPH SMITH, JUN.\nTHE TESTIMONY OF THREE WITNESSES\nBe it known unto all nations, kindreds, tongues, and people, unto\nwhom this work shall come: That we, through the grace of God the\nFather, and our Lord Jesus Christ, have seen the plates which\ncontain this record, which is a record of the people of Nephi,\nand also of the Lamanites, their brethren, and also of the people\nof Jared, who came from the tower of which hath been s

1945

##Creating an Index

###Selecting our vector store

We are going to use FAISS - facebook AI similarity search for the vector embedding

In [15]:
!pip install -q -U faiss-cpu tiktoken sentence-transformers

now we use the huggingface token to get access to model

In [16]:
from huggingface_hub import notebook_login
notebook_login()

In [17]:
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore


store = LocalFileStore("./cache/")

embed_model_id = 'sentence-transformers/all-MiniLM-L12-v2'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)

vector_store = FAISS.from_documents(text1, embedder)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Now to check if the vector store works properly

In [77]:
query = "Who is the author of this book?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)

neighbour, and knows not that at the same time he divides himself from
the whole body, or corporation. But herein is the gift and mercy of God,
the Author of this society, in that, once cut off we may grow together
and become part of the whole again. But if this happen often the misery
is that the further a man is run in this division, the harder he is to
be reunited and restored again: and however the branch which, once cut
of afterwards was graffed in, gardeners can tell you is not like that
which sprouted together at first, and still continued in the unity of
the body.
VIII. To grow together like fellow branches in matter of good
correspondence and affection; but not in matter of opinions. They that
shall oppose thee in thy right courses, as it is not in their power to
divert thee from thy good action, so neither let it be to divert thee
from thy good affection towards them. But be it thy care to keep thyself
constant in both; both in a right judgment and action, and in true
﻿The Pr

Let's see how much time the CacheBackedEmbeddings pattern saves us:

In [34]:
%%timeit -n 1 -r 1
query = "Who was marcus aurelius?"
embedding_vector = embedder.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4) #retrieving the top 4 documents

26.2 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [35]:
%%timeit
query = "Who was marcus aurelius?"
embedding_vector = embedder.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4) #retrieving the top 4 documents

29.6 ms ± 5.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


###Creating an index trying with another vector store: ChromaDB

In [80]:
!pip install chromadb

In [81]:
from langchain.vectorstores import Chroma

INDEX_PERSIST_DIRECTORY = os.getenv('INDEX_PERSIST_DIRECTORY', "./data/chromadb")

In [82]:
!pip install langchain-chroma

In [18]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_chroma import Chroma

In [19]:
#loading the document
loader = TextLoader("pg17.txt")
documents = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectordbChroma = Chroma.from_documents(
        documents=text1,
        embedding=embeddings
    )


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [20]:
query = "Who is aurelius?"

docs = vectordbChroma.similarity_search(query)

print(docs[0].page_content)

APPENDIX
     GLOSSARY
Original Transcriber's Notes:
This text was scanned by J. Boulton using Textbridge OCR. The Greek
portions of the text have been added by hand and they will require the
standard "Symbol" font "symbol.ttf" to be installed in the system fonts
folder. This is a standard Windows font, so should be present on most
systems. To contact the scanner e-mail: magicjon@ic24.net INTRODUCTION
This is the Plain Text version, see medma10h.txt or .zip for the HTML
version with the various symbols mentioned above.
INTRODUCTION
MARCUS AURELIUS ANTONINUS was born on April 26, A.D. 121. His real name
was M. Annius Verus, and he was sprung of a noble family which claimed
descent from Numa, second King of Rome. Thus the most religious of
emperors came of the blood of the most pious of early kings. His father,
Annius Verus, had held high office in Rome, and his grandfather, of
the same name, had been thrice Consul. Both his parents died young, but


In [86]:
# save to disk
db2 = Chroma.from_documents(docs, embeddings, persist_directory="./chroma_db")
docs = db2.similarity_search(query)

# load from disk
db3 = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)
docs = db3.similarity_search(query)
print(docs[0].page_content)

APPENDIX
     GLOSSARY
Original Transcriber's Notes:
This text was scanned by J. Boulton using Textbridge OCR. The Greek
portions of the text have been added by hand and they will require the
standard "Symbol" font "symbol.ttf" to be installed in the system fonts
folder. This is a standard Windows font, so should be present on most
systems. To contact the scanner e-mail: magicjon@ic24.net INTRODUCTION
This is the Plain Text version, see medma10h.txt or .zip for the HTML
version with the various symbols mentioned above.
INTRODUCTION
MARCUS AURELIUS ANTONINUS was born on April 26, A.D. 121. His real name
was M. Annius Verus, and he was sprung of a noble family which claimed
descent from Numa, second King of Rome. Thus the most religious of
emperors came of the blood of the most pious of early kings. His father,
Annius Verus, had held high office in Rome, and his grandfather, of
the same name, had been thrice Consul. Both his parents died young, but


Similarity search witha  a score and choosing the low score one as it is better(Cosine Score)

In [87]:
docs = db3.similarity_search_with_score(query)
docs[0]

(Document(page_content='APPENDIX\n     GLOSSARY\nOriginal Transcriber\'s Notes:\nThis text was scanned by J. Boulton using Textbridge OCR. The Greek\nportions of the text have been added by hand and they will require the\nstandard "Symbol" font "symbol.ttf" to be installed in the system fonts\nfolder. This is a standard Windows font, so should be present on most\nsystems. To contact the scanner e-mail: magicjon@ic24.net INTRODUCTION\nThis is the Plain Text version, see medma10h.txt or .zip for the HTML\nversion with the various symbols mentioned above.\nINTRODUCTION\nMARCUS AURELIUS ANTONINUS was born on April 26, A.D. 121. His real name\nwas M. Annius Verus, and he was sprung of a noble family which claimed\ndescent from Numa, second King of Rome. Thus the most religious of\nemperors came of the blood of the most pious of early kings. His father,\nAnnius Verus, had held high office in Rome, and his grandfather, of\nthe same name, had been thrice Consul. Both his parents died young, bu

In [38]:
docs = vector_store.similarity_search_with_score(query)
docs[0]

(Document(page_content='He married Lucilla, daughter of M. A., and died 169 A.D.\nVespasian, 9th Roman Emperor XENOCRATES of Chalcedon, 396-314 B.C., a\nphilosopher, and president of the Academy.\nEnd of the Project Gutenberg EBook of Meditations, by Marcus Aurelius\n*** END OF THIS PROJECT GUTENBERG EBOOK MEDITATIONS ***\n***** This file should be named 2680-8.txt or 2680-8.zip *****\nThis and all associated files of various formats will be found in:\n        http://www.gutenberg.org/2/6/8/2680/\nProduced by J. Boulton\nUpdated editions will replace the previous one--the old editions\nwill be renamed.\nCreating the works from public domain print editions means that no\none owns a United States copyright in these works, so the Foundation\n(and you!) can copy and distribute it in the United States without\npermission and without paying copyright royalties.  Special rules,\nset forth in the General Terms of Use part of this license, apply to\ncopying and distributing Project Gutenberg-tm

In [39]:
docs = vectordbChroma.similarity_search_with_score(query)
docs[0]

(Document(page_content='APPENDIX\n     GLOSSARY\nOriginal Transcriber\'s Notes:\nThis text was scanned by J. Boulton using Textbridge OCR. The Greek\nportions of the text have been added by hand and they will require the\nstandard "Symbol" font "symbol.ttf" to be installed in the system fonts\nfolder. This is a standard Windows font, so should be present on most\nsystems. To contact the scanner e-mail: magicjon@ic24.net INTRODUCTION\nThis is the Plain Text version, see medma10h.txt or .zip for the HTML\nversion with the various symbols mentioned above.\nINTRODUCTION\nMARCUS AURELIUS ANTONINUS was born on April 26, A.D. 121. His real name\nwas M. Annius Verus, and he was sprung of a noble family which claimed\ndescent from Numa, second King of Rome. Thus the most religious of\nemperors came of the blood of the most pious of early kings. His father,\nAnnius Verus, had held high office in Rome, and his grandfather, of\nthe same name, had been thrice Consul. Both his parents died young, bu

It seems the FAISS has a better similarity score than the chroma

#3.Building the retrieval chain

###Creating a prompt template

In [40]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Use the following context to answer the user's query. If you cannot answer the question, please respond with 'I don't know'.

Question:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [1]:
  !pip install huggingface-hub -q

In [2]:

from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install accelerate transformers
import os
os._exit(00)

In [1]:
import torch

import transformers
# from transformers import fuyu

model_id = "meta-llama/Llama-2-13b-chat-hf"

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_id
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Lla

In [2]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id
)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [50]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=True,
    temperature=1.0,
    max_new_tokens=100,

)

In [45]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

In [46]:
retriever = vector_store.as_retriever()

In [47]:
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()

qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    callbacks=[handler],
    return_source_documents=True
)

Testing

In [49]:
qa_with_sources_chain({"query" : "Who was aurelius?"})



> Entering new RetrievalQA chain...


ValueError: The following `model_kwargs` are not used by the model: ['response'] (note: typos in the generate arguments will also show up in this list)

now lets check from Chromadb if there is a difference

In [29]:
retriever = vectordbChroma.as_retriever()

In [30]:
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()

qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    callbacks=[handler],
    return_source_documents=True
)

Testing

In [32]:
qa_with_sources_chain({"query" : "Who was aurelius antoninus?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Who was aurelius antoninus?',
 'result': 'Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\n1844; (6) George Long, 1862; (7) G. H. Rendall, 1898; and (8) J.\nJackson, 1906. Renan\'s "Marc-Aurèle"--in his "History of the Origins of\nChristianity," which appeared in 1882--is the most vital and original\nbook to be had relating to the time of Marcus Aurelius. Pater\'s "Marius\nthe Epicurean" forms another outside commentary, which is of service in\nthe imaginative attempt to create again the period.\nMARCUS AURELIUS ANTONINUS THE ROMAN EMPEROR\nHIS FIRST BOOK\nconcerning HIMSELF:\nWherein Antoninus recordeth, What and of whom, whether Parents, Friends,\nor Masters; by their good examples, or good advice and counsel, he had\nlearned:\nDivided into Numbers or Sections.\nANTONINUS Book vi. Num. xlviii. Whensoever thou wilt rejoice thyself,\nthink and meditate upon

In [36]:
qa_with_sources_chain({"query" : "Which year was aurelius born in?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Which year was aurelius born in?',
 'result': 'Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\nAPPENDIX\n     GLOSSARY\nOriginal Transcriber\'s Notes:\nThis text was scanned by J. Boulton using Textbridge OCR. The Greek\nportions of the text have been added by hand and they will require the\nstandard "Symbol" font "symbol.ttf" to be installed in the system fonts\nfolder. This is a standard Windows font, so should be present on most\nsystems. To contact the scanner e-mail: magicjon@ic24.net INTRODUCTION\nThis is the Plain Text version, see medma10h.txt or .zip for the HTML\nversion with the various symbols mentioned above.\nINTRODUCTION\nMARCUS AURELIUS ANTONINUS was born on April 26, A.D. 121. His real name\nwas M. Annius Verus, and he was sprung of a noble family which claimed\ndescent from Numa, second King of Rome. Thus the most religious of\nemperors c

In [37]:
qa_with_sources_chain({"query" : "Who is pikachu?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Who is pikachu?',
 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\ncolour: russata (red), albata (white), veneta (blue), prasina (green).\nThere was high rivalry between them, and riots and bloodshed not\ninfrequently.\nCithaeron, a mountain range N. of Attica.\nComedy, ancient; a term applied to the Attic comedy of Aristophanes and\nhis time, which criticised persons and politics, like a modern comic\njournal, such as Punck. See New Comedy.\nCompendious, short.\nConceit, opinion.\nContentation, contentment.\nCrates, a Cynic philosopher of the 4th century B.C.\nCroesus, King of Lydia, proverbial for wealth; he reigned 560-546 B.C.\nCynics, a school of philosophers, founded by Antisthenes. Their texts\nwere a kind of caricature of Socraticism. Nothing was good but virtue,\nnothing bad but vice. The Cynics repudiated all civil and social claims,\nand